In [9]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pylab as plt
from dmba import plotDecisionTree, classificationSummary, regressionSummary
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import xgboost as xgb


import pandas as pd
from sklearn.model_selection import train_test_split


from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
  
# data (as pandas dataframes) 
x = breast_cancer_wisconsin_original.data.features 
y = breast_cancer_wisconsin_original.data.targets 
  
# metadata 
print(breast_cancer_wisconsin_original.metadata) 
  
# variable information 
print(breast_cancer_wisconsin_original.variables) 


{'uci_id': 15, 'name': 'Breast Cancer Wisconsin (Original)', 'repository_url': 'https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original', 'data_url': 'https://archive.ics.uci.edu/static/public/15/data.csv', 'abstract': 'Original Wisconsin Breast Cancer Database', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 699, 'num_features': 9, 'feature_types': ['Integer'], 'demographics': [], 'target_col': ['Class'], 'index_col': ['Sample_code_number'], 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1990, 'last_updated': 'Sun Mar 10 2024', 'dataset_doi': '10.24432/C5HP4Z', 'creators': ['WIlliam Wolberg'], 'intro_paper': None, 'additional_info': {'summary': "Samples arrive periodically as Dr. Wolberg reports his clinical cases. The database therefore reflects this chronological grouping of the data. This grouping information appears immediately below, having been removed fro

Fill NA

In [3]:
from sklearn.impute import KNNImputer
data_c = x.copy()
imputer = KNNImputer(n_neighbors=25)
data_imputed = imputer.fit_transform(data_c)
data_imputed=pd.DataFrame(data_imputed,columns=data_c.columns)
data_imputed.isna().sum()

Clump_thickness                0
Uniformity_of_cell_size        0
Uniformity_of_cell_shape       0
Marginal_adhesion              0
Single_epithelial_cell_size    0
Bare_nuclei                    0
Bland_chromatin                0
Normal_nucleoli                0
Mitoses                        0
dtype: int64

In [4]:
x=data_imputed

Split the data

In [5]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25, random_state=42)

Initiate the XGBoost Classifier

In [12]:
xgb_model = xgb.XGBClassifier(random_state=42)

In [13]:
#Fit the Model
xgb_model.fit(x_train,y_train)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got [2 4]

In [15]:
y.head(10)

,Class
0,2
1,2
2,2
3,2
4,2
5,4
6,2
7,2
8,2
9,2


Replace values to correctly run the xg boost

In [16]:
y = y.replace({2:0 , 4:1})

In [19]:
y.head(10)

,Class
0,0
1,0
2,0
3,0
4,0
5,1
6,0
7,0
8,0
9,0


retrain the model

In [20]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25, random_state=42)

Rerun the xgboost

In [21]:
#Fit the Model
xgb_model.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

make predictions

In [22]:
y_pred = xgb_model.predict(x_test)

Evaluate the model

In [24]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9542857142857143
Confusion Matrix:
 [[115   3]
 [  5  52]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.97       118
           1       0.95      0.91      0.93        57

    accuracy                           0.95       175
   macro avg       0.95      0.94      0.95       175
weighted avg       0.95      0.95      0.95       175

